In [4]:
import requests
import csv
import os
from requests import Response
from requests.exceptions import RequestException

URL_DOUBAN_TOP250 = 'https://api.douban.com/v2/movie/top250?apikey=0df993c66c0c636e29ecbb5344252a4a&start=%s'
CURRENT_FILE_PATH = os.path.dirname(os.path.abspath('__file__')) + '/' + 'day2_source' 
CSV_FILE_PATH = CURRENT_FILE_PATH + '/' + 'douban_top250.csv'
IMAGE_FILE_PATH = CURRENT_FILE_PATH
# 250
MAX_PAGE = 20


# 将数据写入csv
def save_data_to_csv(moive_info: [], rank: int):
    moive_info.insert(0, rank)
    with open(CSV_FILE_PATH,"a+") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(moive_info)
        
# csv文件先写入默认行 用w覆盖
def config_default_csv():
    # Checking if a File Exists
    # os.path.isfile('./file.txt')    # True

    # Checking if a Directory Exists
    # os.path.isdir('./file.txt')    # False

    # Checking if Either Exist
    # os.path.exists('./file.txt')    # True

    # 如果资源文件不存在 先创建
    if not os.path.exists(CURRENT_FILE_PATH):
        try:
            os.mkdir(CURRENT_FILE_PATH)
        except OSError as error:
            print('资源文件创建失败 % s' % error)
        else:
            print('资源文件创建成功')

    with open(CSV_FILE_PATH,"w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['排名', "id", "电影名", "评分", '主演', '封面'])

# 把封面图写入本地
def save_image_to_local(info: [], rank: int):
    file_name = '%s-%s.jpg' % (rank, info[0])
    file_url = info[4][:-3] + 'jpg'
    file_path = IMAGE_FILE_PATH + '/' + file_name
    ir = requests.get(file_url, stream=True)
    if ir.status_code == 200:
        with open(file_path, 'wb') as f:
            for chunk in ir:
                f.write(chunk)
    
# 从某一个rank开始获取数据
def get_data(rank: int):
    try:
        request = requests.get(URL_DOUBAN_TOP250 % rank)
    except RequestException as error:
        print('Request error')
    else:
        response = request.json()
        return response.get('subjects')

# 从一个电影dict读取想要的信息
def read_data_from(movie: dict):
    id_str = movie.get('id')
    title = movie.get('title')
    average = movie.get('rating').get('average')
    casts = ''
    arr_casts = movie.get('casts')
    for cast in arr_casts:
        casts += cast.get('name')
        casts += ', '
    casts = casts[:-2]
    image = movie.get('images').get('small')
    return [id_str, title, average, casts, image]

# 开始获取数据
def startRead():
    RANK = 0
    while RANK < MAX_PAGE:
        arr_movies = get_data(RANK)
        if not arr_movies:
            print('请求接口失败')
            return
        #遍历当前20个电影中的每一个
        for movie_info in arr_movies:
            info_list = read_data_from(movie_info)
            RANK += 1       
            save_image_to_local(info_list, RANK)
            save_data_to_csv(info_list, RANK)

def run():
        config_default_csv()
        startRead()
      
run()